# Test Valuation Engines

This notebook walks through:

1. Loading historical SOFR fixings
2. Building a simple two-index yield curve
3. Valuing various products (bullet, IBOR cashflow, OIS cashflow, futures, swaps)

---

## 1. Imports & Setup

In [1]:
import sys, os

# compute the parent of tests → FixedIncomeLib
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

print("Added to sys.path:", repo_root)

Added to sys.path: c:\Users\neels\OneDrive\Desktop\Capstone_Project\FixedIncomeLib


In [2]:
import pandas as pd
from date import Date
from yield_curve import YieldCurve
from valuation import IndexManager
from valuation import ValuationEngineRegistry
from product import (
    ProductBulletCashflow,
    ProductIborCashflow,
    ProductOvernightIndexCashflow,
    ProductFuture,
    ProductRfrFuture,
    ProductIborSwap,
    ProductOvernightSwap,
    ProductPortfolio )

## 2. Load SOFR Fixings

In [3]:
IndexManager.instance()

## 3. Build Dummy Yield Curve

In [4]:
DUMMY_DATA = [
    ["SOFR-1B",            "1M",  0.0020],
    ["SOFR-1B",            "3M",  0.0025],
    ["SOFR-1B",            "6M",  0.0030],
    ["USD-LIBOR-BBA-3M",   "1M",  0.0040],
    ["USD-LIBOR-BBA-3M",   "3M",  0.00425],
    ["USD-LIBOR-BBA-3M",   "6M",  0.00450],
]
build_methods = [
    {"TARGET": "SOFR-1B",          "INTERPOLATION METHOD": "PIECEWISE_CONSTANT"},
    {"TARGET": "USD-LIBOR-BBA-3M", "INTERPOLATION METHOD": "PIECEWISE_CONSTANT"},
]
# Construct DataFrame and YieldCurve as of 2025-06-27
df_curve = pd.DataFrame(DUMMY_DATA, columns=["INDEX", "AXIS1", "VALUES"])
yc = YieldCurve("2025-06-27", df_curve, build_methods)
print("Curve components:", yc.components.keys())

Curve components: dict_keys(['SOFR-1B', 'USD-LIBOR-BBA-3M'])



## 4. Value Bullet Cashflow


In [5]:
# Bullet cashflow PV
bullet = ProductBulletCashflow("2028-05-25", "USD", 1_000_000, "LONG")
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B"},
    bullet
)
ve.calculateValue()
print("Bullet CF PV:", ve.value)

Bullet CF PV: ['USD', np.float64(991474.0537980911)]


## 5. Value IBOR Cashflow

In [6]:
ibor_cf = ProductIborCashflow(
    startDate="2025-06-27",
    endDate="2025-09-27",
    index="USD-LIBOR-BBA-3M",
    spread=0.0,
    notional=1_000_000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(yc, {}, ibor_cf)
ve.calculateValue()
print("Ibor CF PV:", ve.value)

Ibor CF PV: ['USD', np.float64(-1019.824662745844)]


## 6. Value OIS Cashflow

In [7]:
ois_cf = ProductOvernightIndexCashflow(
    effectiveDate="2025-06-27",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="COMPOUND",
    spread=0.0,
    notional=1_000_000,
    longOrShort="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"valuation_date": Date("2025-07-01")},
    ois_cf
)
ve.calculateValue()
print("OIS CF PV:", ve.value)

OIS CF PV: ['USD', np.float64(561.0250872598588)]


## 7. Value Futures

In [8]:
# IBOR Future PV
future = ProductFuture(
    effectiveDate="2025-06-27",
    index="USD-LIBOR-BBA-3M",
    strike=99.5,
    notional=1_000_000,
    longOrShort="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(yc, {}, future)
ve.calculateValue()
print("Future PV:", ve.value)

Future PV: ['USD', np.float64(92070.13490166105)]


In [9]:
# RFR Future PV
rfr_future = ProductRfrFuture(
    effectiveDate="2025-06-27",
    termOrEnd="3M",
    index="SOFR-1B",
    compounding="AVERAGE",
    strike=99.7,
    notional=1_000_000,
    longOrShort="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"valuation_date": Date("2025-07-01")},
    rfr_future
)
ve.calculateValue()
print("RFR Future PV:", ve.value)

RFR Future PV: ['USD', np.float64(-243897.49127401217)]


## 8. Value a Portfolio

In [10]:
# Simple portfolio of bullet + IBOR CF
portfolio = ProductPortfolio([bullet, ibor_cf], weights=[0.5, 0.5])
ve = ValuationEngineRegistry().new_valuation_engine(
    yc,
    {"FUNDING INDEX": "SOFR-1B", "valuation_date": Date("2025-07-01")},
    portfolio
)
ve.calculateValue()
print("Portfolio PV:", ve.value)

Portfolio PV: ['USD', np.float64(990454.2291353453)]


## 9. Value Swaps (IBOR & OIS)

In [11]:
# IBOR Swap
ibor_swap = ProductIborSwap(
    effectiveDate="2025-06-27",
    maturityDate="2026-06-27",
    frequency="6M",
    iborIndex="USD-LIBOR-BBA-3M",
    spread=0.0,
    fixedRate=0.015,
    notional=1_000_000,
    position="SHORT"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ibor_swap
)
ve.calculateValue()
print("IBOR Swap PV:", ve.value, "Par Rate:", ve.parRate())

IBOR Swap PV: ['USD', np.float64(-2000126.8763746223)] Par Rate: 32.41980827800543


In [12]:
# OIS Swap
ois_swap = ProductOvernightSwap(
    effectiveDate="2025-06-27",
    maturityDate="2026-06-27",
    frequency="6M",
    overnightIndex="SOFR-1B",
    spread=0.0,
    fixedRate=0.015,
    notional=1_000_000,
    position="LONG"
)
ve = ValuationEngineRegistry().new_valuation_engine(
    yc, {"FUNDING INDEX": "SOFR-1B"}, ois_swap
)
ve.calculateValue()
print("OIS Swap PV:", ve.value, "Par Rate:", ve.parRate())

OIS Swap PV: ['USD', np.float64(1999949.24423121)] Par Rate: 31.084772494647435
